In [13]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

from data_util import *

from data_generator import DataGenerator

import model as model_util

from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt

from numpy import array
import numpy as np



In [14]:
def reset_seed():
    seed_value= 0
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    import random
    random.seed(seed_value)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    import numpy as np
    np.random.seed(seed_value)
    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    import tensorflow as tf
    tf.random.set_seed(seed_value)

reset_seed()

path = "./"
model_path = path + "stock.h5"

In [37]:
trainX, trainY, positiveX, positiveY, negativeX, negativeY = load_data("ethusd", "Train", path, balanced = True)
valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = load_data("btcusd", "Val", path, balanced = False)

In [38]:
dim = 50
seqs = 30
dropout = 0.1
lays = 5
lays_seq = 1

In [39]:
# https://machinelearningmastery.com/how-to-use-the-timeseriesgenerator-for-time-series-forecasting-in-keras/
# define generator
batch_size = 512

train_generator = get_gen(trainX, trainY, shuffle=True, time_steps = seqs)

val_generator = get_gen(valX, valY, shuffle=False, time_steps = seqs)

x_train, y_train = create_dataset(trainX, trainY, seqs)

x_val, y_val = create_dataset(valX, valY, seqs)


# number of samples
print('Samples: %d' % len(train_generator))


print(x_train.shape)
print(y_train.shape)

features = x_train.shape[-1]

print("Feature: {}".format(features))


Samples: 463
(29616, 30, 21)
(29616,)
Feature: 21


In [40]:
#model.summary()
import tensorflow as tf


use_regular = False #@param {type:"boolean"}

checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             load_weights_on_restart=True)

callbacks_list = []

epochs = 50

if (use_regular):
    print("Training on: {} val {}".format(len(y_train), len(y_val)))
    history = model.fit(
        x=x_train,
        y=y_train,
        validation_data=(x_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
        callbacks=callbacks_list
    )
else:
    print("Training on: {} val {}".format(len(train_generator)*batch_size, len(val_generator)))
    history = model.fit(
        x=train_generator,
        validation_data = val_generator,
        #steps_per_epoch = len(train_generator),
        epochs=epochs,
        verbose=1,
        callbacks=callbacks_list
    )


Training on: 237056 val 24
Epoch 1/50
463/463 [==============================] - 262s 565ms/step - loss: 0.7443 - accuracy: 0.5245 - val_loss: 0.8711 - val_accuracy: 0.3159
Epoch 2/50
463/463 [==============================] - 262s 567ms/step - loss: 0.6967 - accuracy: 0.5812 - val_loss: 1.0448 - val_accuracy: 0.2254
Epoch 3/50
463/463 [==============================] - 263s 568ms/step - loss: 0.6660 - accuracy: 0.6169 - val_loss: 1.1555 - val_accuracy: 0.2280
Epoch 4/50
463/463 [==============================] - 264s 570ms/step - loss: 0.6390 - accuracy: 0.6482 - val_loss: 1.2290 - val_accuracy: 0.2584
Epoch 5/50
218/463 [=============>................] - ETA: 2:17 - loss: 0.6132 - accuracy: 0.6775

KeyboardInterrupt: 

In [ ]:
def show_loss(history): 
    plt.plot(history.history['loss'])
    if (history.history['val_loss']):
        plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

show_loss(history)

In [ ]:
def show_accuracy(history):
    plt.plot(history.history['accuracy'])
    if (history.history['val_accuracy']):
        plt.plot(history.history['val_accuracy'])
    plt.title('accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

show_accuracy(history)

In [ ]:


#model_loaded = load_model(filepath = filepath_encoder)
model_loaded = model

to_int = np.vectorize(lambda x: int(round(x)))

In [35]:
posX_, posY_ = get_gen(valX, valY, shuffle=True, batch_size=2000, time_steps = seqs)[0]

print(positiveX.shape)
print(negativeX.shape)


(37, 21)
(735, 21)


In [36]:
model_loaded.predict(posX_)

array([[0.52532625],
       [0.4931125 ],
       [0.56294477],
       ...,
       [0.5240735 ],
       [0.49755993],
       [0.52340907]], dtype=float32)

In [ ]:
pos_final = to_int(model_loaded.predict(posX_))
neg_final = to_int(model_loaded.predict(negX_))


pos_final

In [ ]:
model_loaded.evaluate(val_generator)

In [ ]:
def check(list):
    pos_count = 0
    neg_count = 0
    for x in list:
        if (x == 1):
            pos_count += 1
        else:
            neg_count += 1
    print("Pos: {} Neg {}".format(pos_count, neg_count))

In [ ]:
print("Pos")
check(pos_final)
print("Neg")
check(neg_final)


In [ ]:
model_loaded.predict(posX_)[:10]

In [ ]:
model_loaded.predict(negX_)[:10]

In [ ]:
model_loaded = model
valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = load_data("btcusd", "Val", path, False)
model_loaded.evaluate(get_gen(valX, valY, shuffle=False, time_steps = seqs)) 

valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = load_data("ethusd", "Val", path, False)
model_loaded.evaluate(get_gen(valX, valY, shuffle=False, time_steps = seqs)) 

valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = load_data("btcusd17", "Val", path, False)
model_loaded.evaluate(get_gen(valX, valY, shuffle=False, time_steps = seqs)) 

valX, valY, positiveX_, positiveY_, negativeX_, negativeY_ = load_data("btcusdAug19", "Val", path, False)
model_loaded.evaluate(get_gen(valX, valY, shuffle=False, time_steps = seqs)) 


In [ ]:
X = list()
Y = list()
X = [x+1 for x in range(200)]
Y = [y for y in X]

In [ ]:
generator = get_gen(X, Y, shuffle=True, batch_size=20)

x, y = create_dataset(X, Y, seqs)

index = 100

In [11]:
#Model 

from tensorflow.keras.layers import BatchNormalization, GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras import datasets, layers, models

import tensorflow as tf

dim = 600

def add_deep_layers(input_layer, units = dim):
    x = Dense(units)(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(units, activation='tanh')(x)
    x = Dropout(0.2)(x)
    x = Dense(units, activation='swish')(x)
    return x

def get_model(seqs, features):
    reset_seed()
    inputX = Input(shape=(
        seqs,
        features
        )
    )
    x = inputX
    x = LSTM(units=dim, return_sequences=True)(x)
    x = add_deep_layers(x, dim)
    x = LSTM(dim, return_sequences=False)(x)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, dim)
    x = add_deep_layers(x, 20)
    x = Dense(10)(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[inputX], outputs=x)
    
    return model

    
def load_model(filepath = "stock.h5"):
    model_loaded = tf.keras.models.load_model(filepath)
    return model_loaded

In [ ]:
def add_deep_layers_custom(input_layer, drop, units):
    x = BatchNormalization()(input_layer)
    x = Dropout(drop)(x)
    x = Dense(units, activation='tanh')(x)
    x = Dropout(drop)(x)
    x = Dense(units, activation='swish')(x)
    return x


def get_model_h(seqs, features):
    reset_seed()
    inputX = Input(shape=(
        seqs,
        features
        )
    )
    x = Dense(dim)(inputX)
    x = BatchNormalization()(x)
    x = LSTM(units=dim, return_sequences=True)(x)
    for lay in range(int(lays_seq)):
        x = add_deep_layers(x, dropout, dim)
    x = LSTM(dim, return_sequences=False)(x)
    
    for lay in range(int(lays)):
        x = add_deep_layers(x, dropout, dim)
    
    x = add_deep_layers(x, dropout, dim)
    x = Dense(10)(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[inputX], outputs=x)
    
    return model
    

In [12]:


#model = load_model(filepath = "drive/My Drive/model/stock.h5")

#!rm "drive/My Drive/model/encoder.h5"

model = get_model(seqs, features)

model.compile(
    optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 30, 21)]          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 600)           1492800   
_________________________________________________________________
dense_40 (Dense)             (None, 30, 600)           360600    
_________________________________________________________________
batch_normalization_12 (Batc (None, 30, 600)           2400      
_________________________________________________________________
dropout_24 (Dropout)         (None, 30, 600)           0         
_________________________________________________________________
dense_41 (Dense)             (None, 30, 600)           360600    
_________________________________________________________________
dropout_25 (Dropout)         (None, 30, 600)           0   

In [ ]:
!rm "stock.h5"